# developing external testting tricks?

## the OpenAI/BigCode variant only works on UNix

## I know that tempdir and subprocess works and does timeout... but can you actually recover from that?
should I allow `wgpu-shadertoy` to take a code arg in cli? instead of just
this will only work for singlepass shaders just now.

wgpu-22 gott shader compilation info (not in wgpu-native yet)

In [3]:
import os
import tempfile
import subprocess


file_template = """
from wgpu_shadertoy import Shadertoy

shader_code = '''{}'''

shader = Shadertoy(shader_code, shader_type="glsl", offscreen=True)

if __name__ == "__main__":
    frame = shader.snapshot(123.45)
    frame2 = shader.snapshot(678.90)
    # shader.show()
"""

def run_shader_in_subprocess(shader_code:str, timeout:float=10) -> str:
    """
    writes the shadercode into a temporary file, and tries to run the shader with a snapshot. This will catch any kind of errors or panics. Even the really bad ones.
    a timeout can be specified. But the enumerate adapter is rather slow, so it can take over 5 seconds just to do that on a slow computer. Therefore a timeout in the range of 10 seconds is needed to avoid false detections.
    returns either "ok", "timeout" or "error"
    #TODO: not tested on unix systems, might required a change in the python command to call.
    """
    status = "ok" # default case
    with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False, encoding="utf-8") as f:
        f.write(file_template.format(shader_code))
        f.flush()
        try:
            p = subprocess.run(["python", f.name], capture_output=True, timeout=timeout) # this might not work as expect on Linux ...
            
        except (Exception, RuntimeError) as e:
            if isinstance(e, subprocess.TimeoutExpired):
                status = "timeout"
            else:
                status = "other validation error"
    
    # cleanup temp file, delete_on_close was only added in Python 3.12?
    os.remove(f.name)
        
    msg = p.stderr.decode("utf-8")
    if status == "ok":
        # print(f"{p.returncode=}")
        if p.returncode != 0 or msg:
            if "panic" in msg:
                status = "panic"
                # print(msg)
            else:
                status = "error via return code or msg"
            
    return status, msg


In [2]:
new_code = """
void mainImage( out vec4 fragColor, in vec2 fragCoord )
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.xy;

    // Time varying pixel color
    vec3 col = 0.5 + 0.5*cos(iTime+uv.xyx+vec3(0,2,4));

    // Output to screen
    fragColor = vec4(col,1.0);
}
"""


error_code = """
void mainImage( out vec4 fragColor, in vec2 fragCoord )
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.xy;

    // Time varying pixel color
    vec3 col = 0.5 + 0.5*cos(iTime+uv.xyx+vec3(0,2,4));

    // Output to screen
    fragColor = vec4(coll,1.0);
}
"""

# this panics because it loses device, works in 22.1!
# new variant that panics (or not?)
minimal_code = """
void mainImage( out vec4 fragColor, in vec2 fragCoord ) {

    vec3 col = vec3(0.0);
    for (float i = 0.0; i < 10.0; i -= 1.0) {
        col += vec3(0.2);
    }
    fragColor = vec4(col, 1.0);
}
"""

panic_code2 = """
float bar(inout float a, float b){
    a *= 2.0;
    return a + b;
}

void mainImage( out vec4 fragColor, in vec2 fragCoord )
{
    float foo = 0.1;
    vec2 foo2 = vec2(0.2, 0.3);
    float foo3 = bar(foo2.x, foo);
    
    // Output to screen
    vec3 col = vec3(foo3, foo2.x, foo2.y);
    fragColor = vec4(col,1.0);
}
"""
test_codes = [new_code, error_code, minimal_code, panic_code2]

In [ ]:
seconds = 10

for code in test_codes:
    status, msg = run_shader_in_subprocess(code, timeout=seconds)
    print(f"{status=}, {msg=}")
    print("-"*40)

In [ ]:
# Goal is to run shaders quickly by an unsafe worker.
# if the worker panics or timesout, it should be noted and a new worker spawned.
# to communicate with the worker, we can send the shader_code as a string via a pipe/file?
# the worker should return the ok/error result via stdout (the pipe).
# if the worker is still good, we clear the communication pipe and send the next shader.

worker_script = """
import sys
import wgpu_shadertoy
import time
from wgpu.utils.device import get_default_device

def run_shader(shader_code:str, timeout:float=10) -> str:
    try:
        shader = wgpu_shadertoy.Shadertoy(shader_code, shader_type="glsl", offscreen=True)
        frame = shader.snapshot(123.45)
        frame2 = shader.snapshot(678.90)
        return "ok"
    except Exception as e:
        return "error"

if __name__ == "__main__":
    # this is the slow part, should only run once per worker
    get_default_device()
    while True:
        temp_shader_file = sys.stdin.readline().strip()
        if not temp_shader_file:
            break
        with open(temp_shader_file, "r") as f:
            shader_code = f.read()
        result = run_shader(shader_code)
        sys.stdout.write(result + "\\n")
        sys.stdout.flush()
        time.sleep(1.1) # wait for the next file?
"""

def start_worker():
    with subprocess.Popen(["python", "-u", "-c", worker_script], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True) as p:
        print("worker started")
        


In [6]:
import subprocess
import time


toy_worker = """
import sys
import time

def toy(name) -> str:
    if name == "3":
        time.sleep(2.5) # simulate a timeout
        return "oh, it's 3"
    return "hi\t" + name

if __name__ == "__main__":
    while True:
        msg = sys.stdin.readline().strip()
        if not msg:
            break
        result = toy(msg)
        sys.stdout.write(result + "\\n")
        sys.stdout.flush()
"""


def start_toy_worker():
    with subprocess.Popen(["python", "-u", "-c", toy_worker], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True) as p:
        print("toy worker started")
        for i in range(5):
            print(i)
            # stdout, _ = p.communicate(str(i) + "\n", timeout=2.0)
            # print(stdout.strip())
            # p.poll()
            p.stdin.write(str(i) + "\n")
            p.stdin.flush()
            print(p.stdout.readline().strip())
            time.sleep(0.1)
        p.stdin.close()
        print("toy worker done")


In [7]:
start_toy_worker()

toy worker started
0
hi	0
1
hi	1
2
hi	2
3
oh, it's 3
4
hi	4
toy worker done
